# Análisis Exploratorio de Datos (EDA) - Online Retail

En este cuaderno, realizaremos un **análisis paso a paso** de un dataset de facturación que contiene columnas como:
 - **InvoiceNo**: Número de factura
 - **StockCode**: Código del producto
 - **Description**: Descripción del producto
 - **Quantity**: Cantidad vendida (puede ser negativa en devoluciones)
 - **InvoiceDate**: Fecha de la factura
 - **UnitPrice**: Precio unitario de cada artículo
 - **CustomerID**: ID del cliente (puede haber nulos si no se registró)
 - **Country**: País donde se realizó la compra

El objetivo es **entender** el comportamiento de las ventas, los países más relevantes, los productos más comprados, y otras métricas de interés.

In [1]:
!wget https://github.com/javierherrera1996/lecture_analytics/raw/main/datasets/ecomerce_customers_segmentation.zip
!unzip ecomerce_customers_segmentation.zip

--2025-09-11 00:05:55--  https://github.com/javierherrera1996/lecture_analytics/raw/main/datasets/ecomerce_customers_segmentation.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/lecture_analytics/main/datasets/ecomerce_customers_segmentation.zip [following]
--2025-09-11 00:05:55--  https://raw.githubusercontent.com/javierherrera1996/lecture_analytics/main/datasets/ecomerce_customers_segmentation.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7548686 (7.2M) [application/zip]
Saving to: ‘ecomerce_customers_segmentation.zip’

ecomerce_customers_ 100%[===================>]

## 1. Importar librerías y cargar datos
Asegúrate de tener el archivo CSV en la misma carpeta, o ajusta la ruta de `pd.read_csv` según tu necesidad.

In [2]:
import pandas as pd
import numpy as np
# Carga del dataset (ajusta el nombre a tu archivo)
df = pd.read_csv('/content/data.csv',encoding="latin-1") # Puedes necesitar encoding='latin1' si hay caracteres especiales
pd.set_option('display.max_columns', None)
# Mostramos las primeras filas para inspección
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## 2. Información general del dataset
Aquí veremos:
- Dimensiones del DataFrame
- Tipos de datos
- Presencia de valores nulos
- Estadísticos descriptivos de las columnas numéricas

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [ ]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [ ]:
# Estadísticos descriptivos
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


**Observación**: Algunas facturas pueden ser devoluciones si `Quantity` es negativa. Además, puede haber filas sin `CustomerID`.

## 3. Limpieza básica y transformación
### 3.1. Revisar y eliminar posibles filas con datos problemáticos
 - Filas donde `Description` esté vacía
 - Filas con `Quantity == 0` (puede haber poco sentido en un registro así)
 - Filas con `UnitPrice <= 0` (podría ser error o promoción anómala)
 - Convertir la columna `InvoiceDate` a formato fecha, para luego analizar tiempos

In [ ]:
df.dropna(subset=['Description'])

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [ ]:
df.isnull().sum()

,0
InvoiceNo,0
StockCode,0
Description,0
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,133626
Country,0


In [ ]:
df = df[df['Quantity'] !=0]

In [ ]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [5]:
df = df[df['UnitPrice']>0]

In [9]:
type(df["InvoiceDate"][0])

str

In [15]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], format="MM/DD/YYYY HH:MM")

In [16]:
df["InvoiceDate"][0]

Timestamp('2010-12-01 08:26:00')

## 4. Creación de columna "TotalPrice"
Para conocer el valor total de cada línea de venta, calculamos `Quantity * UnitPrice`.

In [17]:
df['total'] = df['Quantity'] * df['UnitPrice']

In [20]:
type(df['total'][0])

numpy.float64

## 5. Análisis paso a paso
A continuación, veremos distintos análisis que **paso a paso** nos ayudarán a entender mejor el negocio.

### 5.1. ¿Cuántas transacciones totales hay y cuántos clientes diferentes?
Esto da un primer vistazo al tamaño del negocio en términos de facturas y base de clientes.

In [21]:
df.groupby('CustomerID').count()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country,TotalPrice,total
CustomerID,,,,,,,,,
12346.0,2,2,2,2,2,2,2,2,2
12347.0,182,182,182,182,182,182,182,182,182
12348.0,31,31,31,31,31,31,31,31,31
12349.0,73,73,73,73,73,73,73,73,73
12350.0,17,17,17,17,17,17,17,17,17
...,...,...,...,...,...,...,...,...,...
18280.0,10,10,10,10,10,10,10,10,10
18281.0,7,7,7,7,7,7,7,7,7
18282.0,13,13,13,13,13,13,13,13,13


### 5.2. ¿Cuáles son los países más frecuentes en las ventas?
Podemos contar cuántas líneas de factura vienen de cada país. Esto ayuda a entender nuestro alcance geográfico.

In [22]:
df.groupby('Country').count()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,TotalPrice,total
Country,,,,,,,,,
Australia,1256,1256,1256,1256,1256,1256,1256,1256,1256
Austria,401,401,401,401,401,401,401,401,401
Bahrain,19,19,19,19,19,19,17,19,19
Belgium,2069,2069,2069,2069,2069,2069,2069,2069,2069
Brazil,32,32,32,32,32,32,32,32,32
Canada,151,151,151,151,151,151,151,151,151
Channel Islands,758,758,758,758,758,758,758,758,758
Cyprus,622,622,622,622,622,622,622,622,622
Czech Republic,30,30,30,30,30,30,30,30,30


> Tip: Si queremos ver el total en **monetario** por país, podemos agrupar por `Country` y sumar `TotalPrice`.

In [27]:
df.groupby('Country')['TotalPrice'].sum()

,TotalPrice
Country,
Australia,137077.270
Austria,10154.320
Bahrain,548.400
Belgium,40910.960
Brazil,1143.600
Canada,3666.380
Channel Islands,20086.290
Cyprus,12946.290
Czech Republic,707.720


### 5.3. ¿Cuáles son los productos más vendidos?
Aquí vamos a usar `Description` para ver cuántas líneas se han vendido (o la suma total de `Quantity`).

In [29]:
# Top 10 productos por cantidad total vendida

df.groupby('Description')['Quantity'].sum().sort_values(ascending=False).head(10)


,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53847
JUMBO BAG RED RETROSPOT,47359
ASSORTED COLOUR BIRD ORNAMENT,36381
POPCORN HOLDER,36334
PACK OF 72 RETROSPOT CAKE CASES,36039
WHITE HANGING HEART T-LIGHT HOLDER,35313
RABBIT NIGHT LIGHT,30680
MINI PAINT SET VINTAGE,26437
PACK OF 12 LONDON TISSUES,26111


### 5.4. ¿Cuál es el total de ingresos (revenue) y el ticket promedio?
 - El ingreso total es la suma de **TotalPrice**.
 - El ticket promedio podemos calcularlo como el promedio de **TotalPrice** dentro de cada factura (o línea).

In [30]:
df.groupby('InvoiceNo')['TotalPrice'].sum()

,TotalPrice
InvoiceNo,
536365,139.12
536366,22.20
536367,278.73
536368,70.05
536369,17.85
...,...
C581484,-168469.60
C581490,-32.53
C581499,-224.69


> Para un ticket promedio a nivel de **factura**, debemos agrupar por `InvoiceNo`, sumar `TotalPrice` y luego promediar.

In [32]:
# Ticket promedio por factura
df.groupby('InvoiceNo')['TotalPrice'].sum().mean

<bound method Series.mean of InvoiceNo
536365        139.12
536366         22.20
536367        278.73
536368         70.05
536369         17.85
             ...    
C581484   -168469.60
C581490       -32.53
C581499      -224.69
C581568       -54.75
C581569        -7.50
Name: TotalPrice, Length: 23796, dtype: float64>

### 5.6. ¿Tenemos devoluciones?
Como `Quantity` puede ser negativa, **representa** devoluciones. Veamos cuántas hay y el impacto en valor.

In [35]:
# Filtrar devoluciones (Quantity < 0)
df_devoluciones = df[df['Quantity'] < 0]

In [36]:
print(df_devoluciones)

       InvoiceNo StockCode                       Description  Quantity  \
141      C536379         D                          Discount        -1   
154      C536383    35004C   SET OF 3 COLOURED  FLYING DUCKS        -1   
235      C536391     22556    PLASTERS IN TIN CIRCUS PARADE        -12   
236      C536391     21984  PACK OF 12 PINK PAISLEY TISSUES        -24   
237      C536391     21983  PACK OF 12 BLUE PAISLEY TISSUES        -24   
...          ...       ...                               ...       ...   
540449   C581490     23144   ZINC T-LIGHT HOLDER STARS SMALL       -11   
541541   C581499         M                            Manual        -1   
541715   C581568     21258        VICTORIAN SEWING BOX LARGE        -5   
541716   C581569     84978  HANGING HEART JAR T-LIGHT HOLDER        -1   
541717   C581569     20979     36 PENCILS TUBE RED RETROSPOT        -5   

               InvoiceDate  UnitPrice  CustomerID         Country  TotalPrice  \
141    2010-12-01 09:41:00    

### 5.7. ¿Cuántos clientes realizan compras repetidas?
Podemos ver cuántos clientes compran más de una vez.

In [37]:
df.groupby('CustomerID')['InvoiceNo'].count()

,InvoiceNo
CustomerID,
12346.0,2
12347.0,182
12348.0,31
12349.0,73
12350.0,17
...,...
18280.0,10
18281.0,7
18282.0,13


## 6. Posibles siguientes pasos
- **Crear segmentaciones** (p. ej. RFM: Recency, Frequency, Monetary) para clasificar clientes.
- **Analizar descuentos/promos** si tuvieras columnas extra.
- **Fusionar** con datos de marketing para calcular CAC (Customer Acquisition Cost) y LTV (Lifetime Value), etc.
- **Explorar** variables estacionales o días de la semana con mayor venta.

Con este **análisis paso a paso**, se obtiene un primer panorama de:
 - Cantidad de facturas y clientes.
 - Productos más vendidos.
 - Ingresos por país.
 - Montos totales y tickets promedio.
 - Distribución mensual de ventas.
 - Devoluciones y recurrencia de clientes.

¡A medida que se adquieran más habilidades en Python y análisis de datos, se pueden añadir gráficos, segmentaciones avanzadas y modelos predictivos!